#### Basic LLMChain

In [1]:
from langchain import PromptTemplate, LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [2]:
prompt = PromptTemplate.from_template("What is the capital of {country}?")

In [3]:
llm   = ChatOpenAI()

chain = LLMChain(llm   = llm, 
                 prompt= prompt)

C:\Users\bhupe\AppData\Local\Temp\ipykernel_19712\1546732881.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm   = llm,


example

In [4]:
# Run the chain
result = chain.invoke({"country": "France"})
print(result)  # Output should be "Paris"

{'country': 'France', 'text': 'The capital of France is Paris.'}


#### Simple Sequential Chain

In [5]:
from langchain.chains import SimpleSequentialChain

In [32]:
llm_model = 'gpt-4o-mini'

In [33]:
llm = ChatOpenAI(temperature=0.1, 
                 model      =llm_model)

In [34]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [35]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [36]:
overall_simple_chain = SimpleSequentialChain(chains =[chain_one, chain_two],
                                             verbose=True
                                            )

In [43]:
product = "Defender 90"

In [44]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
A suitable name for a company that specializes in making Defender 90 vehicles could be "Defender Dynamics" or "Defender Innovations." These names convey a sense of expertise and focus on the Defender 90 model while suggesting a commitment to quality and innovation. Other options could include "Defender Craftworks" or "Defender Performance Engineering," depending on the specific focus of the company.
Defender Dynamics specializes in crafting high-quality Defender 90 vehicles, blending innovation and expertise for exceptional performance and durability.

> Finished chain.


'Defender Dynamics specializes in crafting high-quality Defender 90 vehicles, blending innovation and expertise for exceptional performance and durability.'

#### Sequential Chain

In [45]:
from langchain.chains.sequential import SequentialChain

In [47]:
prompt1 = PromptTemplate.from_template("What is the capital of {country}?")
prompt2 = PromptTemplate.from_template("Describe the main attractions in {city}.")

In [48]:
llm = ChatOpenAI()

In [49]:
chain1 = LLMChain(llm       = llm, 
                  prompt    = prompt1, 
                  output_key="city")               # Sets "city" as output for chain1

chain2 = LLMChain(llm       = llm, 
                  prompt    = prompt2, 
                  output_key= "city_description")  # Sets "city_description" as output for chain2

In [50]:
sequential_chain = SequentialChain(
    chains           = [chain1, chain2],
    input_variables  = ["country"],          # Input to the first chain
    output_variables = ["city_description"]  # Output from the last chain
)

In [51]:
result = sequential_chain.invoke({"country": "Japan"})
print(result["city_description"])  # Should describe attractions in the capital city

Tokyo, the capital of Japan, is a vibrant and bustling metropolis that offers a wide range of attractions for visitors to enjoy. Some of the main attractions in Tokyo include:

1. The iconic Tokyo Tower, a symbol of the city and one of its most famous landmarks. Visitors can take in panoramic views of the city from the observation decks.

2. The historic Senso-ji Temple in Asakusa, one of Tokyo's oldest and most significant temples. The temple is known for its stunning architecture and bustling Nakamise shopping street.

3. The bustling Shibuya Crossing, one of the busiest pedestrian intersections in the world. Visitors can experience the energy and excitement of Tokyo's vibrant Shibuya district.

4. The upscale shopping district of Ginza, known for its luxury boutiques, department stores, and fine dining establishments.

5. The peaceful Meiji Shrine, located in the heart of Tokyo's bustling Harajuku district. Visitors can explore the tranquil grounds and participate in traditional Shi

Example

In [52]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm       = llm, 
                     prompt    = first_prompt, 
                     output_key= "English_Review"
                    )


In [53]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm       = llm, 
                     prompt    = second_prompt, 
                     output_key= "summary"
                    )

In [54]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm       = llm, 
                       prompt    = third_prompt,
                       output_key= "language"
                      )


In [55]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm       = llm, 
                      prompt    = fourth_prompt,
                      output_key= "followup_message"
                     )

In [56]:
overall_chain = SequentialChain(
    chains          = [chain_one, chain_two, chain_three, chain_four],
    input_variables = ["Review"],
    output_variables= ["English_Review", "summary","followup_message"],
    verbose         = True
)

In [57]:
review = '''
বাংলাদেশ একটি সমৃদ্ধ ইতিহাস ও সংস্কৃতির দেশ। এই দেশে রয়েছে প্রাচীন স্থাপত্য, নদী, সবুজ প্রকৃতি এবং অসংখ্য জাতিগোষ্ঠীর বৈচিত্র্য। 
বাংলাদেশের মানুষ অতিথিপরায়ণ এবং তারা তাদের ঐতিহ্য ও সংস্কৃতিকে ভালোবাসে। প্রতি বছর এখানে বিভিন্ন ধরনের উৎসব পালন করা হয়, 
যেমন পহেলা বৈশাখ, ঈদ, দুর্গাপূজা, এবং বিজয় দিবস। এই উৎসবগুলোতে মানুষ একসঙ্গে আনন্দ করে, গান গায়, নাচে এবং খাবার খায়।

বাংলাদেশের অন্যতম বড় আকর্ষণ হচ্ছে সুন্দরবন, যা পৃথিবীর বৃহত্তম ম্যানগ্রোভ বন। এখানে রয়েল বেঙ্গল টাইগার, চিত্রা হরিণ, 
বানর এবং নানা ধরনের বন্যপ্রাণীর বসবাস। সুন্দরবনের প্রকৃতি অত্যন্ত সুন্দর এবং এটি প্রতিবছর হাজার হাজার পর্যটককে আকর্ষণ করে। 
তাছাড়া কক্সবাজার বিশ্বের দীর্ঘতম সমুদ্রসৈকত হিসাবে পরিচিত। সাদা বালুর সৈকত, নীল পানি, এবং সূর্যাস্তের দৃশ্য প্রতিটি দর্শকের মনে দাগ কাটে।

বাংলাদেশের মানুষের জীবনধারা খুবই সরল এবং প্রকৃতির সাথে গভীরভাবে সংযুক্ত। বেশিরভাগ মানুষ গ্রামাঞ্চলে বাস করে এবং তারা 
কৃষিকাজে ব্যস্ত থাকে। ধান, পাট, চা, এবং মাছ বাংলাদেশের প্রধান পণ্য। দেশটি ধীরে ধীরে শিল্প এবং প্রযুক্তিতে এগিয়ে যাচ্ছে এবং এশিয়ার 
একটি গুরুত্বপূর্ণ অর্থনৈতিক কেন্দ্রে পরিণত হচ্ছে।
'''

In [58]:
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': '\nবাংলাদেশ একটি সমৃদ্ধ ইতিহাস ও সংস্কৃতির দেশ। এই দেশে রয়েছে প্রাচীন স্থাপত্য, নদী, সবুজ প্রকৃতি এবং অসংখ্য জাতিগোষ্ঠীর বৈচিত্র্য। \nবাংলাদেশের মানুষ অতিথিপরায়ণ এবং তারা তাদের ঐতিহ্য ও সংস্কৃতিকে ভালোবাসে। প্রতি বছর এখানে বিভিন্ন ধরনের উৎসব পালন করা হয়, \nযেমন পহেলা বৈশাখ, ঈদ, দুর্গাপূজা, এবং বিজয় দিবস। এই উৎসবগুলোতে মানুষ একসঙ্গে আনন্দ করে, গান গায়, নাচে এবং খাবার খায়।\n\nবাংলাদেশের অন্যতম বড় আকর্ষণ হচ্ছে সুন্দরবন, যা পৃথিবীর বৃহত্তম ম্যানগ্রোভ বন। এখানে রয়েল বেঙ্গল টাইগার, চিত্রা হরিণ, \nবানর এবং নানা ধরনের বন্যপ্রাণীর বসবাস। সুন্দরবনের প্রকৃতি অত্যন্ত সুন্দর এবং এটি প্রতিবছর হাজার হাজার পর্যটককে আকর্ষণ করে। \nতাছাড়া কক্সবাজার বিশ্বের দীর্ঘতম সমুদ্রসৈকত হিসাবে পরিচিত। সাদা বালুর সৈকত, নীল পানি, এবং সূর্যাস্তের দৃশ্য প্রতিটি দর্শকের মনে দাগ কাটে।\n\nবাংলাদেশের মানুষের জীবনধারা খুবই সরল এবং প্রকৃতির সাথে গভীরভাবে সংযুক্ত। বেশিরভাগ মানুষ গ্রামাঞ্চলে বাস করে এবং তারা \nকৃষিকাজে ব্যস্ত থাকে। ধান, পাট, চা, এবং মাছ বাংলাদেশের প্রধান পণ্য। দেশটি ধীরে ধীরে শিল্প এবং প্রযু

#### Router chains

In [59]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""

math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""

computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [72]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [73]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [74]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [75]:
destination_chains = {}

for p_info in prompt_infos:
    name            = p_info["name"]
    prompt_template = p_info["prompt_template"]
    
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    
    chain  = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [65]:
destination_chains

{'physics': LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template="You are a very smart physics professor. You are great at answering questions about physics in a conciseand easy to understand manner. When you don't know the answer to a question you admitthat you don't know.\n\nHere is a question:\n{input}"), additional_kwargs={})]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000014BAB87C0B0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000014BAB784500>, root_client=<openai.OpenAI object at 0x0000014BACF6CC80>, root_async_client=<openai.AsyncOpenAI object at 0x0000014BAB87C920>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}

In [76]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [77]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a 
language model select the model prompt best suited for the input. 

You will be given the names of the available prompts and a 
description of what the prompt is best suited for. 

You may also revise the original input if you think that revising
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string  name of the prompt to use or "DEFAULT"
    "next_inputs": string  a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [78]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

print(router_template)

Given a raw text input to a 
language model select the model prompt best suited for the input. 

You will be given the names of the available prompts and a 
description of what the prompt is best suited for. 

You may also revise the original input if you think that revising
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string  name of the prompt to use or "DEFAULT"
    "next_inputs": string  a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
physics: Good for answering questions about physics
math: Good for answering math questions
Histor

In [79]:
router_prompt = PromptTemplate(
    template       = router_template,
    input_variables= ["input"],
    output_parser  = RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [80]:
chain = MultiPromptChain(router_chain      =router_chain, 
                         destination_chains=destination_chains, 
                         default_chain     =default_chain, 
                         verbose           =True
                        )

In [81]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation refers to the electromagnetic radiation emitted by an idealized object known as a "black body," which absorbs all incident radiation, regardless of frequency or angle. A perfect black body is a theoretical concept that does not reflect or transmit any light, making it appear completely black at room temperature.\n\nThe key characteristics of black body radiation include:\n\n1. **Temperature Dependence**: The amount and spectrum of radiation emitted by a black body depend solely on its temperature. As the temperature increases, the intensity of the emitted radiation increases, and the peak wavelength shifts to shorter wavelengths (this is described by Wien\'s displacement law).\n\n2. **Planck\'s Law**: Max Planck derived a formula that describes the spectral distribution of radiation emitted by a black body at a given temperature. This law was pivotal in the development of quantum mechanics.\n\n3. **Stefan-Boltzmann Law**: The total energy radiated per unit surface

In [83]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'To solve the problem \\(2 + 2\\), we can break it down into its component parts:\n\n1. Identify the numbers involved: We have the numbers 2 and 2.\n2. Understand the operation: The operation we are performing is addition.\n\nNow, we can combine the two numbers:\n\n\\[\n2 + 2 = 4\n\\]\n\nSo, the answer to the question \\(2 + 2\\) is \\(4\\).'

#### Simple Memory Chain
- provision memory to the chains

In [84]:
from langchain.memory import SimpleMemory

In [85]:
llm = ChatOpenAI()

In [87]:
memory = SimpleMemory()

In [88]:
user_input = "Name the Country where Bangalore city is."

In [89]:
# Access the memory to get previous interactions
previous_memory = memory.load_memory_variables({"input": user_input})  # This accesses the stored memory directly

# Construct the context from previous interactions
context = ""
for key, value in previous_memory.items():
    context += f"User: {key}\nBot: {value}\n"

context

''

In [90]:
# Define a prompt template that uses both current input and memory context
prompt_template = PromptTemplate.from_template(
    f'''You are a helpful assistant.  
    
     Here are the previous interactions: {context}
     
     User: {user_input}
     Bot:
     '''
)

prompt_template

PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant.  \n    \n     Here are the previous interactions: \n     \n     User: Name the Country where Bangalore city is.\n     Bot:\n     ')

In [91]:
chain = LLMChain(llm   = llm, 
                 prompt= prompt_template)

In [92]:
combined_input = {"context": context.strip(), "input": user_input}
combined_input

{'context': '', 'input': 'Name the Country where Bangalore city is.'}

In [93]:
# Get response from the chain
response = chain.invoke(combined_input)

In [94]:
# Display the response
print("Bot:", response)

Bot: {'context': '', 'input': 'Name the Country where Bangalore city is.', 'text': 'Bangalore city is located in India.'}


In [95]:
memory.save_context({"input": user_input}, {"output": response})

In [96]:
memory.json()

'{"memories":{}}'

In [97]:
memory.to_json()

{'lc': 1,
 'type': 'not_implemented',
 'id': ['langchain', 'memory', 'simple', 'SimpleMemory'],
 'repr': 'SimpleMemory()'}

#### ConversationBufferMemory

In [98]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

In [99]:
llm = ChatOpenAI()

In [100]:
memory = ConversationBufferMemory()

C:\Users\bhupe\AppData\Local\Temp\ipykernel_19712\995385594.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [101]:
# Get user input
user_input = "Where is Bangalore?"

In [102]:
# Load previous memory (initially empty)
previous_memory = memory.load_memory_variables({})  # No input needed for initial load

In [103]:
previous_memory.get('history', "")

''

In [104]:
# Create a prompt using context and user input
prompt_template = PromptTemplate.from_template(
    f'''You are a helpful assistant.  
    
    Here are the previous interactions: {context.strip()}
    
    User: {user_input}
    Bot:
    '''
)

In [105]:
chain = LLMChain(llm=llm, prompt=prompt_template)

In [106]:
combined_input = {"context": context.strip(), "input": user_input}
response = chain.invoke(combined_input)

In [107]:
print("Bot:", response)

Bot: {'context': '', 'input': 'Where is Bangalore?', 'text': 'Bangalore is located in the southern part of India, in the state of Karnataka. It is known for being a major technology hub and is often referred to as the "Silicon Valley of India".'}


In [108]:
memory.save_context({"input": user_input}, {"output": str(response)})

In [109]:
import json

# Directly pretty-print the dictionary with indentation
print(json.dumps(memory.to_json(), indent=4))

{
    "lc": 1,
    "type": "not_implemented",
    "id": [
        "langchain",
        "memory",
        "buffer",
        "ConversationBufferMemory"
    ],
    "repr": "ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Where is Bangalore?', additional_kwargs={}, response_metadata={}), AIMessage(content='{\\'context\\': \\'\\', \\'input\\': \\'Where is Bangalore?\\', \\'text\\': \\'Bangalore is located in the southern part of India, in the state of Karnataka. It is known for being a major technology hub and is often referred to as the \"Silicon Valley of India\".\\'}', additional_kwargs={}, response_metadata={})]))"
}


In [110]:
# Load previous memory (initially empty)
context = dict(memory.load_memory_variables({}))

In [111]:
context

{'history': 'Human: Where is Bangalore?\nAI: {\'context\': \'\', \'input\': \'Where is Bangalore?\', \'text\': \'Bangalore is located in the southern part of India, in the state of Karnataka. It is known for being a major technology hub and is often referred to as the "Silicon Valley of India".\'}'}

In [112]:
context['history']

'Human: Where is Bangalore?\nAI: {\'context\': \'\', \'input\': \'Where is Bangalore?\', \'text\': \'Bangalore is located in the southern part of India, in the state of Karnataka. It is known for being a major technology hub and is often referred to as the "Silicon Valley of India".\'}'

In [113]:
memory.buffer

'Human: Where is Bangalore?\nAI: {\'context\': \'\', \'input\': \'Where is Bangalore?\', \'text\': \'Bangalore is located in the southern part of India, in the state of Karnataka. It is known for being a major technology hub and is often referred to as the "Silicon Valley of India".\'}'

In [114]:
memory.chat_memory

InMemoryChatMessageHistory(messages=[HumanMessage(content='Where is Bangalore?', additional_kwargs={}, response_metadata={}), AIMessage(content='{\'context\': \'\', \'input\': \'Where is Bangalore?\', \'text\': \'Bangalore is located in the southern part of India, in the state of Karnataka. It is known for being a major technology hub and is often referred to as the "Silicon Valley of India".\'}', additional_kwargs={}, response_metadata={})])

In [115]:
memory.dict()

{'chat_memory': {'messages': [{'content': 'Where is Bangalore?',
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'human',
    'name': None,
    'id': None},
   {'content': '{\'context\': \'\', \'input\': \'Where is Bangalore?\', \'text\': \'Bangalore is located in the southern part of India, in the state of Karnataka. It is known for being a major technology hub and is often referred to as the "Silicon Valley of India".\'}',
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'ai',
    'name': None,
    'id': None}]},
 'output_key': None,
 'input_key': None,
 'return_messages': False,
 'human_prefix': 'Human',
 'ai_prefix': 'AI',
 'memory_key': 'history'}

In [117]:
mem_dict = memory.dict()

In [118]:
mem_dict['chat_memory']

{'messages': [{'content': 'Where is Bangalore?',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'human',
   'name': None,
   'id': None},
  {'content': '{\'context\': \'\', \'input\': \'Where is Bangalore?\', \'text\': \'Bangalore is located in the southern part of India, in the state of Karnataka. It is known for being a major technology hub and is often referred to as the "Silicon Valley of India".\'}',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'ai',
   'name': None,
   'id': None}]}

In [119]:
mem_dict['chat_memory']['messages']

[{'content': 'Where is Bangalore?',
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'human',
  'name': None,
  'id': None},
 {'content': '{\'context\': \'\', \'input\': \'Where is Bangalore?\', \'text\': \'Bangalore is located in the southern part of India, in the state of Karnataka. It is known for being a major technology hub and is often referred to as the "Silicon Valley of India".\'}',
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'ai',
  'name': None,
  'id': None}]

In [120]:
mem_dict['chat_memory']['messages'][0]['content']

'Where is Bangalore?'

In [121]:
mem_dict['chat_memory']['messages'][1]['content']

'{\'context\': \'\', \'input\': \'Where is Bangalore?\', \'text\': \'Bangalore is located in the southern part of India, in the state of Karnataka. It is known for being a major technology hub and is often referred to as the "Silicon Valley of India".\'}'

| Feature             | `LLMChain`                        | `ConversationChain`                     |
|---------------------|-----------------------------------|-----------------------------------------|
| **Purpose**         | Single interactions, stateless tasks | Multi-turn conversations, context maintenance |
| **Structure**       | Prompt + LLM                       | Prompt + LLM + Conversation history    |
| **State**           | Stateless                          | Stateful (maintains conversation history) |
| **Use Case**        | Text generation, classification, independent Q&A | Chatbots, conversational applications with context |


Example

In [134]:
llm = ChatOpenAI()

In [135]:
prompt_template = PromptTemplate(input_variables= ["context", "input"], 
                                 template       = "{context}\nUser: {input}\nAI:")

In [136]:
chain = LLMChain(llm   = llm, 
                 prompt= prompt_template)

In [137]:
context = ""

In [138]:
user_inputs = ["What is the capital of France?", 
               "And its population?", 
               "What about Germany?",
               "And its population?", 
               "Compare the 2 citites in terms of job opportunities"
              ]

In [139]:
from pprint import pprint

In [140]:
for user_input in user_inputs:
    # Run the LLM chain, adding context
    response = chain.invoke({"context": context, "input": user_input})
    
    print(user_input)
    pprint(response)
    print("-----------------")

    # Update context with user input and LLM response
    context += f"\nUser: {user_input}\nAI: {response}"

What is the capital of France?
{'context': '',
 'input': 'What is the capital of France?',
 'text': 'The capital of France is Paris.'}
-----------------
And its population?
{'context': '\n'
            'User: What is the capital of France?\n'
            "AI: {'context': '', 'input': 'What is the capital of France?', "
            "'text': 'The capital of France is Paris.'}",
 'input': 'And its population?',
 'text': "{'context': '', 'input': 'And its population?', 'text': 'The "
         "population of Paris is approximately 2.2 million.'}"}
-----------------
What about Germany?
{'context': '\n'
            'User: What is the capital of France?\n'
            "AI: {'context': '', 'input': 'What is the capital of France?', "
            "'text': 'The capital of France is Paris.'}\n"
            'User: And its population?\n'
            'AI: {\'context\': "\\nUser: What is the capital of France?\\nAI: '
            "{'context': '', 'input': 'What is the capital of France?', "
          

**Context Management**

- using deque

In [141]:
from collections import deque

In [142]:
llm = ChatOpenAI()

In [143]:
prompt_template = PromptTemplate(input_variables= ["context", "input"], 
                                 template       = "{context}\nUser: {input}\nAI:")

In [144]:
chain = LLMChain(llm   = llm, 
                 prompt= prompt_template)

In [145]:
user_inputs = ["What is the capital of France?", 
               "And its population?", 
               "What about Germany?",
               "Compare the 2 citites in terms of weather in general",
               "Which city is bigger in geographical size?"
              ]

In [146]:
context = deque(maxlen=5)

In [147]:
def format_context(context):

    #return "\n".join(f"User: {user_input}\nAI: {llm_response}" for user_input, llm_response in context)
    """Format the context from the deque into a string."""
    formatted_context = ""
    for user_input, llm_response in context:
        formatted_context += f"User: {user_input}\nAI: {llm_response}\n"
    return formatted_context.strip()

In [148]:
for user_input in user_inputs:
     # Run the LLM chain with the formatted context
    formatted_context = format_context(context)
    
    response = chain.invoke({"context": context, "input": user_input})
    
    print(user_input)
    pprint(response)
    print("-----------------")

    # Update context with the new interaction
    context.append((user_input, response['text']))

What is the capital of France?
{'context': deque([], maxlen=5),
 'input': 'What is the capital of France?',
 'text': 'Paris'}
-----------------
And its population?
{'context': deque([('What is the capital of France?', 'Paris')], maxlen=5),
 'input': 'And its population?',
 'text': 'The population of Paris is approximately 2.1 million people.'}
-----------------
What about Germany?
{'context': deque([('What is the capital of France?', 'Paris'),
                   ('And its population?',
                    'The population of Paris is approximately 2.1 million '
                    'people.')],
                  maxlen=5),
 'input': 'What about Germany?',
 'text': "I'm sorry, I do not have information on Germany at the moment. Is "
         'there anything else I can help you with?'}
-----------------
Compare the 2 citites in terms of weather in general
{'context': deque([('What is the capital of France?', 'Paris'),
                   ('And its population?',
                    'The popu

#### using LIST struc 

In [149]:
from collections import defaultdict

In [150]:
llm = ChatOpenAI()

prompt_template = PromptTemplate(
    input_variables=["context", "input"],
    template="{context}\nUser: {input}\nAI:"
)

In [151]:
chain = LLMChain(llm=llm, prompt=prompt_template)

In [152]:
user_inputs = [
    "What is the capital of France?",
    "And its population?",
    "What about Germany?",
    "Compare the 2 cities in terms of weather in general",
    "Which city is bigger in geographical size?"
]

In [153]:
context = []

In [154]:
def format_context(context):
    """Format the context from the list into a string."""
    formatted_context = ""
    for user_input, llm_response in context:
        formatted_context += f"User: {user_input}\nAI: {llm_response}\n"
    return formatted_context.strip()

In [155]:
for user_input in user_inputs:
    # Run the LLM chain with the formatted context
    formatted_context = format_context(context)

    # Invoke the chain
    response = chain.invoke({"context": formatted_context, "input": user_input})

    print(f"User: {user_input}")
    pprint(response)
    print("-----------------")

    # Update context with the new interaction
    context.append((user_input, response['text']))


User: What is the capital of France?
{'context': '',
 'input': 'What is the capital of France?',
 'text': 'The capital of France is Paris.'}
-----------------
User: And its population?
{'context': 'User: What is the capital of France?\n'
            'AI: The capital of France is Paris.',
 'input': 'And its population?',
 'text': 'The population of Paris is approximately 2.2 million people.'}
-----------------
User: What about Germany?
{'context': 'User: What is the capital of France?\n'
            'AI: The capital of France is Paris.\n'
            'User: And its population?\n'
            'AI: The population of Paris is approximately 2.2 million people.',
 'input': 'What about Germany?',
 'text': 'The capital of Germany is Berlin, and its population is '
         'approximately 3.7 million people.'}
-----------------
User: Compare the 2 cities in terms of weather in general
{'context': 'User: What is the capital of France?\n'
            'AI: The capital of France is Paris.\n'
      

#### Use ConversationBufferMemory

In [156]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [157]:
llm = ChatOpenAI()

In [158]:
memory = ConversationBufferMemory()

In [159]:
chain = ConversationChain(llm     = llm, 
                          #prompt= prompt_template,  
                          memory  = memory, 
                          verbose = True)

C:\Users\bhupe\AppData\Local\Temp\ipykernel_19712\3517037667.py:1: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  chain = ConversationChain(llm     = llm,


In [160]:
user_inputs = [
    "What is the capital of China?",
    "And its population?",
    "What about India?",
    "Compare the 2 cities in terms of weather in general",
    "Which city is bigger in geographical size?",
    "How do the cultures of these places differ?",
    "What languages are predominantly spoken in these locations?",
    "What is the general cost of living like in each place?",
    "How do the healthcare systems compare between these regions?",
    "What environmental challenges do these areas face?",
    "How have these locations changed over the last few decades?"
]

In [161]:
for user_input in user_inputs:
    # Invoke the chain
    response = chain.invoke({"input": user_input})
    
    print(f"User: {user_input}")
    pprint(response)
    print("---------------------------------------------------------------------")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What is the capital of China?
AI:

> Finished chain.
User: What is the capital of China?
{'history': '',
 'input': 'What is the capital of China?',
 'response': 'The capital of China is Beijing. It is a bustling metropolis '
             'with a rich history dating back thousands of years. Beijing is '
             'home to iconic landmarks such as the Forbidden City, Tiananmen '
             'Square, and the Great Wall of China. It is also the political '
             'and cultural center of the country.'}
---------------------------------------------------------------------


> Entering new ConversationChain chain...
Prompt after formatting:
The following is 

#### counting the number of tokens at each turn

In [163]:
from langchain.callbacks import get_openai_callback

In [164]:
llm = ChatOpenAI()

In [165]:
memory = ConversationBufferMemory()

In [167]:
chain = ConversationChain(llm   = llm, 
                          #prompt= prompt_template,  
                          memory= memory)

In [168]:
user_inputs = [
    "What is the capital of China?",
    "And its population?",
    "What about India?",
    "Compare the 2 cities in terms of weather in general",
    "Which city is bigger in geographical size?",
    "How do the cultures of these places differ?",
    "What languages are predominantly spoken in these locations?",
    "What is the general cost of living like in each place?",
    "How do the healthcare systems compare between these regions?",
    "What environmental challenges do these areas face?",
    "How have these locations changed over the last few decades?"
]

In [169]:
for user_input in user_inputs:

    with get_openai_callback() as cb:
        # Invoke the chain
        response = chain.invoke({"input": user_input})
        
    
    print(f"User: {user_input}")
    pprint(response)
    print(f'>>>> Spent a total of {cb.total_tokens} tokens')
    print("---------------------------------------------------------------------")

User: What is the capital of China?
{'history': '',
 'input': 'What is the capital of China?',
 'response': 'The capital of China is Beijing. It is one of the most populous '
             'cities in the world and is known for its rich history, cultural '
             'heritage, and modern developments. Beijing is home to iconic '
             'landmarks such as the Forbidden City, Tiananmen Square, and the '
             'Great Wall of China. It serves as the political, cultural, and '
             'educational center of China.'}
>>>> Spent a total of 141 tokens
---------------------------------------------------------------------
User: And its population?
{'history': 'Human: What is the capital of China?\n'
            'AI: The capital of China is Beijing. It is one of the most '
            'populous cities in the world and is known for its rich history, '
            'cultural heritage, and modern developments. Beijing is home to '
            'iconic landmarks such as the Forbidden

In [170]:
print(chain.memory.buffer)

Human: What is the capital of China?
AI: The capital of China is Beijing. It is one of the most populous cities in the world and is known for its rich history, cultural heritage, and modern developments. Beijing is home to iconic landmarks such as the Forbidden City, Tiananmen Square, and the Great Wall of China. It serves as the political, cultural, and educational center of China.
Human: And its population?
AI: As of 2021, the population of Beijing is estimated to be around 21 million people. It is a bustling metropolis with a diverse population from various regions of China and around the world. The city's population has been growing steadily due to factors such as urbanization, migration, and economic opportunities.
Human: What about India?
AI: The population of India is estimated to be over 1.3 billion people as of 2021, making it the second most populous country in the world after China. India is a diverse country with a rich cultural heritage and a wide range of languages, relig

#### Pros and Cons of Storing Everything in Context

| Pros                                                                 | Cons                                                                     |
|----------------------------------------------------------------------|--------------------------------------------------------------------------|
| Storing everything gives the LLM the maximum amount of information. | More tokens mean slower response times and higher costs.                |
| Storing everything is simple and intuitive.                         | Long conversations cannot be remembered as we hit the LLM token limit (4096 tokens for text-davinci-003 and gpt-3.5-turbo). |
| Enhanced continuity in conversation, leading to more coherent responses. | Risk of overwhelming the model with irrelevant or outdated information, leading to potential confusion. |
| Ability to reference previous interactions verbatim, improving specificity. | Increased complexity in managing context, especially in dynamic or multi-turn conversations. |
| Greater flexibility in handling varied topics and user requests.     | Need for careful context management to avoid exceeding token limits.    |

#### Additional Considerations

- **Memory Management**: As conversations grow longer, it's essential to implement strategies to manage memory effectively. This may involve summarizing earlier parts of the conversation or discarding less relevant context.

- **Cost Efficiency**: Consideration of costs associated with token usage is vital. Implementing a context management system that intelligently selects which parts of the conversation to retain can help optimize both performance and costs.

- **Performance Trade-offs**: While having access to extensive context can enhance the quality of responses, it may also lead to slower processing times. Evaluating the balance between context richness and response latency is crucial, especially in applications requiring real-time interaction.

- **Relevance Filtering**: It can be beneficial to implement filtering mechanisms that prioritize storing relevant interactions or summarizing past information rather than retaining everything verbatim. This approach can help maintain context while staying within token limits.

- **User Control**: Providing users with options to adjust how much context is stored (e.g., through settings for short or long-term memory) can enhance user experience and engagement.


quick example ...

In [171]:
llm_model = "gpt-3.5-turbo"

llm    = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm    =llm, 
    memory = memory,
    verbose=True
)

In [172]:
conversation.predict(input="Hi, describe machine Learning in 25 words")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, describe machine Learning in 25 words
AI:

> Finished chain.


'Machine learning is a subset of artificial intelligence that uses algorithms to analyze data, learn patterns, and make predictions without explicit programming instructions.'

In [173]:
conversation.predict(input="what are evaluation methods?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, describe machine Learning in 25 words
AI: Machine learning is a subset of artificial intelligence that uses algorithms to analyze data, learn patterns, and make predictions without explicit programming instructions.
Human: what are evaluation methods?
AI:

> Finished chain.


'Evaluation methods in machine learning are used to assess the performance of a model, such as accuracy, precision, recall, F1 score, ROC curve, and confusion matrix.'

In [174]:
conversation.predict(input="what are validation methods?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, describe machine Learning in 25 words
AI: Machine learning is a subset of artificial intelligence that uses algorithms to analyze data, learn patterns, and make predictions without explicit programming instructions.
Human: what are evaluation methods?
AI: Evaluation methods in machine learning are used to assess the performance of a model, such as accuracy, precision, recall, F1 score, ROC curve, and confusion matrix.
Human: what are validation methods?
AI:

> Finished chain.


'Validation methods in machine learning are techniques used to assess the generalization ability of a model, such as cross-validation, holdout validation, and leave-one-out validation.'

In [175]:
conversation.predict(input="Hi, my name is Big Giant")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, describe machine Learning in 25 words
AI: Machine learning is a subset of artificial intelligence that uses algorithms to analyze data, learn patterns, and make predictions without explicit programming instructions.
Human: what are evaluation methods?
AI: Evaluation methods in machine learning are used to assess the performance of a model, such as accuracy, precision, recall, F1 score, ROC curve, and confusion matrix.
Human: what are validation methods?
AI: Validation methods in machine learning are techniques used to assess the generalization ability of a model, such as cross-validation, holdout validation, and leave-one-out validation.
Human: Hi, my name i

"Hello Big Giant! It's nice to meet you. How can I assist you today?"

In [176]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, describe machine Learning in 25 words
AI: Machine learning is a subset of artificial intelligence that uses algorithms to analyze data, learn patterns, and make predictions without explicit programming instructions.
Human: what are evaluation methods?
AI: Evaluation methods in machine learning are used to assess the performance of a model, such as accuracy, precision, recall, F1 score, ROC curve, and confusion matrix.
Human: what are validation methods?
AI: Validation methods in machine learning are techniques used to assess the generalization ability of a model, such as cross-validation, holdout validation, and leave-one-out validation.
Human: Hi, my name i

'Your name is Big Giant.'

In [177]:
print(memory.buffer)

Human: Hi, describe machine Learning in 25 words
AI: Machine learning is a subset of artificial intelligence that uses algorithms to analyze data, learn patterns, and make predictions without explicit programming instructions.
Human: what are evaluation methods?
AI: Evaluation methods in machine learning are used to assess the performance of a model, such as accuracy, precision, recall, F1 score, ROC curve, and confusion matrix.
Human: what are validation methods?
AI: Validation methods in machine learning are techniques used to assess the generalization ability of a model, such as cross-validation, holdout validation, and leave-one-out validation.
Human: Hi, my name is Big Giant
AI: Hello Big Giant! It's nice to meet you. How can I assist you today?
Human: What is my name?
AI: Your name is Big Giant.


In [178]:
memory.load_memory_variables({})

{'history': "Human: Hi, describe machine Learning in 25 words\nAI: Machine learning is a subset of artificial intelligence that uses algorithms to analyze data, learn patterns, and make predictions without explicit programming instructions.\nHuman: what are evaluation methods?\nAI: Evaluation methods in machine learning are used to assess the performance of a model, such as accuracy, precision, recall, F1 score, ROC curve, and confusion matrix.\nHuman: what are validation methods?\nAI: Validation methods in machine learning are techniques used to assess the generalization ability of a model, such as cross-validation, holdout validation, and leave-one-out validation.\nHuman: Hi, my name is Big Giant\nAI: Hello Big Giant! It's nice to meet you. How can I assist you today?\nHuman: What is my name?\nAI: Your name is Big Giant."}

#### Conversation Summary Memory

In [179]:
from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain.chains import ConversationChain

In [180]:
llm = ChatOpenAI()

In [181]:
memory = ConversationSummaryMemory(llm=llm)

C:\Users\bhupe\AppData\Local\Temp\ipykernel_19712\1952740995.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)


In [182]:
conversation_sum_chain = ConversationChain(llm   = llm, 
                          #prompt                = prompt_template,  
                          memory                 = memory)

In [183]:
# Example interactions
user_inputs = [
    "What is the capital of China?",
    "And its population?",
    "What about India?",
    "Compare the 2 cities in terms of weather in general",
    "Which city is bigger in geographical size?",
    "How do the cultures of these places differ?",
    "What languages are predominantly spoken in these locations?",
    "What is the general cost of living like in each place?",
    "How do the healthcare systems compare between these regions?",
    "What environmental challenges do these areas face?",
    "How have these locations changed over the last few decades?"
]

In [184]:
for user_input in user_inputs:

    with get_openai_callback() as cb:
        # Invoke the chain
        response = conversation_sum_chain.invoke({"input": user_input})
        
    
    print(f"User: {user_input}")
    pprint(response)
    print(f'>>>> Spent a total of {cb.total_tokens} tokens')
    print("---------------------------------------------------------------------")

User: What is the capital of China?
{'history': '',
 'input': 'What is the capital of China?',
 'response': 'The capital of China is Beijing. It is one of the most populous '
             'cities in the world and serves as the political, economic, and '
             'cultural center of the country. Beijing is known for its '
             'historic landmarks such as the Forbidden City, Tiananmen Square, '
             'and the Great Wall of China.'}
>>>> Spent a total of 391 tokens
---------------------------------------------------------------------
User: And its population?
{'history': 'The human asks what the capital of China is. The AI responds that '
            'the capital is Beijing, which is a populous city serving as the '
            'political, economic, and cultural center of the country. It is '
            'known for landmarks like the Forbidden City, Tiananmen Square, '
            'and the Great Wall of China.',
 'input': 'And its population?',
 'response': 'As of the l

#### Pros and Cons of Summarizing Context in LLM Conversations

| Pros                                                           | Cons                                                                                           |
|----------------------------------------------------------------|------------------------------------------------------------------------------------------------|
| Shortens the number of tokens for long conversations.          | Can result in higher token usage for smaller conversations.                                   |
| Enables much longer conversations.                             | Memorization of the conversation history is wholly reliant on the summarization ability of the intermediate summarization LLM. |
| Relatively straightforward implementation, intuitively simple to understand. | Also requires token usage for the summarization LLM; this increases costs (but does not limit conversation length). |

#### Additional Notes

- **Token Efficiency**: Summarization can be effective for managing long conversations, especially if it reduces the total tokens more than it adds.
- **Dependency on Summarization Quality**: The effectiveness of this approach depends heavily on the summarization model’s ability to capture key details accurately.
- **Cost vs. Length**: While summarization adds costs due to extra token usage, it enables conversations to extend indefinitely without hitting token limits.
- **Implementation Simplicity**: Summarizing the context periodically keeps the implementation relatively straightforward and understandable.

#### Conversation buffer Window memory

In [185]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [187]:
memory = ConversationBufferWindowMemory(k=1)

In [188]:
conversation_window_chain = ConversationChain(llm   = llm, 
                          #prompt                = prompt_template,  
                          memory                 = memory)

In [189]:
user_inputs = [
    "What is the capital of China?",
    "And its population?",
    "What about India?",
    "Compare the 2 cities in terms of weather in general",
    "Which city is bigger in geographical size?",
    "How do the cultures of these places differ?",
    "What languages are predominantly spoken in these locations?",
    "What is the general cost of living like in each place?",
    "How do the healthcare systems compare between these regions?",
    "What environmental challenges do these areas face?",
    "How have these locations changed over the last few decades?"
]

In [190]:
for user_input in user_inputs:

    with get_openai_callback() as cb:
        # Invoke the chain
        response = conversation_window_chain.invoke({"input": user_input})
        
    
    print(f"User: {user_input}")
    pprint(response)
    print(f'>>>> Spent a total of {cb.total_tokens} tokens')
    print("---------------------------------------------------------------------")

User: What is the capital of China?
{'history': '',
 'input': 'What is the capital of China?',
 'response': 'The capital of China is Beijing. It is one of the most populous '
             'cities in the world and has a rich history dating back thousands '
             'of years. Beijing is known for its iconic landmarks such as the '
             'Forbidden City, Tiananmen Square, and the Great Wall of China. '
             'It is also the political and cultural center of the country.'}
>>>> Spent a total of 136 tokens
---------------------------------------------------------------------
User: And its population?
{'history': 'Human: What is the capital of China?\n'
            'AI: The capital of China is Beijing. It is one of the most '
            'populous cities in the world and has a rich history dating back '
            'thousands of years. Beijing is known for its iconic landmarks '
            'such as the Forbidden City, Tiananmen Square, and the Great Wall '
            'of 

#### Conversation buffer token memory

In [191]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI

llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [192]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)

C:\Users\bhupe\AppData\Local\Temp\ipykernel_19712\684848764.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)


#### Stuff chain

- stuff more than 1 documents together at the time of LLM invocation
- Pros & Cons

In [193]:
from langchain.chains import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

from langchain.schema import Document

In [194]:
summary_prompt = PromptTemplate(input_variables= ["text"], 
                                template       = "Summarize the following text: {text}")

In [195]:
llm_chain = LLMChain(llm=llm, 
                     prompt=summary_prompt)

In [196]:
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain)

C:\Users\bhupe\AppData\Local\Temp\ipykernel_19712\4194767714.py:1: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  stuff_chain = StuffDocumentsChain(llm_chain=llm_chain)


In [197]:
# Define the documents (text chunks) to be summarized as instances of Document
documents = [
    Document(page_content="Artificial Intelligence (AI) has a profound impact on many industries, including healthcare and finance."),
    Document(page_content="Machine learning is a subset of AI that focuses on developing systems that can learn from data."),
    Document(page_content="Deep learning, a more complex form of machine learning, has enabled advancements in image and speech recognition.")
]

In [198]:
summary = stuff_chain.invoke(input={"input_documents": documents})
summary

{'input_documents': [Document(metadata={}, page_content='Artificial Intelligence (AI) has a profound impact on many industries, including healthcare and finance.'),
  Document(metadata={}, page_content='Machine learning is a subset of AI that focuses on developing systems that can learn from data.'),
  Document(metadata={}, page_content='Deep learning, a more complex form of machine learning, has enabled advancements in image and speech recognition.')],
 'output_text': 'AI has had a significant impact on various industries such as healthcare and finance. Machine learning, a subset of AI, is focused on developing systems that can learn from data. Deep learning, a more advanced form of machine learning, has led to advancements in image and speech recognition.'}

#### Refine Chain

In [200]:
from langchain.chains import RefineDocumentsChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

from langchain.schema import Document

In [201]:
initial_prompt = PromptTemplate(
    input_variables= ["page_content"], 
    template       = "Generate a brief overview based on the following information: {page_content}"
)

In [203]:
refine_prompt = PromptTemplate(
    input_variables = ["existing_answer", "new_information"], 
    template        = "Refine the following answer by incorporating this new information: {new_information}. Answer: {existing_answer}"
)

In [204]:
# Create an initial LLMChain for the first response
initial_chain = LLMChain(llm=llm, prompt=initial_prompt)

# Create a refine LLMChain for the refinement process
refine_chain = LLMChain(llm=llm, prompt=refine_prompt)

In [205]:
# Combine the chains into a RefineDocumentsChain
refine_documents_chain = RefineDocumentsChain(
    initial_llm_chain    = initial_chain,       # Initial processing chain
    refine_llm_chain     = refine_chain,        # Refinement processing chain
    initial_response_name= "initial_response",  # Name for the initial response variable
    verbose              = True
)

C:\Users\bhupe\AppData\Local\Temp\ipykernel_19712\987537964.py:2: LangChainDeprecationWarning: This class is deprecated. Please see the migration guide here for a recommended replacement: https://python.langchain.com/docs/versions/migrating_chains/refine_docs_chain/
  refine_documents_chain = RefineDocumentsChain(


In [206]:
documents = [
    Document(page_content="Artificial Intelligence (AI) is a field that simulates human intelligence."),
    Document(page_content="Machine learning is a significant branch of AI that focuses on data analysis."),
    Document(page_content="Deep learning models, a subset of machine learning, utilize neural networks.")
]

In [207]:
initial_response = initial_chain.invoke(input={"page_content": " ".join(doc.page_content for doc in documents)})

In [208]:
print("Initial Response:", initial_response)

Initial Response: {'page_content': 'Artificial Intelligence (AI) is a field that simulates human intelligence. Machine learning is a significant branch of AI that focuses on data analysis. Deep learning models, a subset of machine learning, utilize neural networks.', 'text': 'Artificial Intelligence (AI) is a field that aims to simulate human intelligence in machines. Machine learning, a significant branch of AI, focuses on data analysis and pattern recognition to make predictions and decisions. Deep learning models, a subset of machine learning, utilize neural networks to process and learn from large amounts of data. These models have shown great success in various applications such as image and speech recognition, natural language processing, and autonomous driving.'}


In [210]:
existing_answer = initial_response['text']  # Directly use the 'text' key for the refinement
existing_answer

'Artificial Intelligence (AI) is a field that aims to simulate human intelligence in machines. Machine learning, a significant branch of AI, focuses on data analysis and pattern recognition to make predictions and decisions. Deep learning models, a subset of machine learning, utilize neural networks to process and learn from large amounts of data. These models have shown great success in various applications such as image and speech recognition, natural language processing, and autonomous driving.'

In [211]:
new_information = "AI technologies are rapidly advancing and are applied in various industries."

In [212]:
refine_inputs = {
    "existing_answer": existing_answer,  # Use the initial response output
    "new_information": new_information
}

In [213]:
final_refined_response = refine_chain.invoke(input=refine_inputs)

In [214]:
print("Final Refined Summary:", final_refined_response)

Final Refined Summary: {'existing_answer': 'Artificial Intelligence (AI) is a field that aims to simulate human intelligence in machines. Machine learning, a significant branch of AI, focuses on data analysis and pattern recognition to make predictions and decisions. Deep learning models, a subset of machine learning, utilize neural networks to process and learn from large amounts of data. These models have shown great success in various applications such as image and speech recognition, natural language processing, and autonomous driving.', 'new_information': 'AI technologies are rapidly advancing and are applied in various industries.', 'text': 'Artificial Intelligence (AI) is a rapidly advancing field that aims to simulate human intelligence in machines. Machine learning, a significant branch of AI, focuses on data analysis and pattern recognition to make predictions and decisions. Deep learning models, a subset of machine learning, utilize neural networks to process and learn from 

| Feature                | Stuff Chain                                             | Refine Chain                                           |
|------------------------|--------------------------------------------------------|-------------------------------------------------------|
| **What They Are**      | A chain that combines multiple documents into a single input for the model, typically used for generating a summary or response based on all combined content. | A chain designed to improve or refine an initial output by incorporating new information iteratively. |
| **Advantages**         | - Simplicity: Easy to implement and understand.<br>- Efficient for generating a response based on a larger context by combining documents. | - Incremental Improvement: Allows for step-by-step enhancement of the output.<br>- Flexibility: Can adapt the existing output with additional context or new information. |
| **Disadvantages**      | - Potential Information Loss: Important details may get lost when summarizing multiple documents into one.<br>- Limited Iteration: Does not allow for iterative feedback on individual documents. | - Complexity: More complex to implement and understand due to multiple steps.<br>- Dependency on Initial Output: Relies on the quality of the initial response, which can affect the final output. |
| **When to Use**       | - When needing to create a summary or response based on multiple pieces of information at once.<br>- Suitable for scenarios where a quick overview is needed without the need for refinement. | - When there is a need to refine or improve an existing response.<br>- Best used when new information becomes available that should enhance an already generated output. |